In [1]:
import dotenv, os, torch
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from main import query_pinecone_from_embedding, query_controller, merge_audio

device = "cuda" if torch.cuda.is_available() else "cpu"
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"), environment="production")
pc_index = pc.Index("hackathon24s")
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-l").to(device)


You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [2]:
embedding = query_controller("I'd like to learn more about Google and Gemini", model)
pc_response = query_pinecone_from_embedding(embedding, pc_index)
custom_pod_s3_url = await merge_audio(pc_response)
print(custom_pod_s3_url)


[('128', 0.797247, 'a16z', 'Sign off'), ('53', 0.740022838, 'Norges Bank Investment Management', 'Introduction'), ('139', 0.73606962, 'a16z', 'Call to action for builders, researchers, and developers'), ('47', 0.732804894, 'The Logan Bartlett Show', 'The Impact of Artificial Intelligence in Business'), ('111', 0.722399414, 'a16z', "Big Tech's involvement in political process")]
